In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
output_notebook()
import numpy as np
import pandas as pd
from main import Atmosphere, justdoit
import astropy.units as u

Loading BokehJS ...

In [2]:
a = Atmosphere(['Cr'])

a.get_gravity(gravity=746.00, gravity_unit=u.Unit('cm/(s**2)'))
a.get_pt( filename='../temp.dat',skiprows=3, delim_whitespace=True, header=None, names=["ind","pressure","temperature","kzz"])
a.get_kz()
justdoit(a, directory='/Users/natashabatalha/Documents/eddysed/input/optics/')



746.0 2.2 0.00010319348393942818 0.00026182519326509597 1485.1892884925815 559812.0538 7.15


ValueError: f(a) and f(b) must have different signs